In [30]:
## Generate a graph
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")

# parameters

n = 30
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

kmax = 4
kmin = 1

fk = k->(2 .*μ*k)^(-k)
fp = harmonicMean

Ω_dict = Dict{Array{Int64, 1}, Float64}()

for k = kmin:kmax, p in partitions(k)
    Ω_dict[p] = fk(sum(p))*fp(p)/10
end

Ω = buildΩ(Ω_dict; by_size=true)

Ω (generic function with 1 method)

In [32]:
## Sample
H = sampleSBM(Z, ϑ, Ω; kmax=kmax, kmin = kmin)

hypergraph
  E: Dict{Integer,Dict}
  D: Array{Integer}((30,)) Integer[9, 4, 2, 4, 6, 1, 5, 3, 3, 4  …  7, 6, 4, 6, 3, 3, 4, 2, 1, 5]


In [33]:
## Run Louvains, Naive, and Faster version
include("jl/hypergraph_louvain.jl")

@time Zlou = Naive_HyperLouvain(H,Ω)
@time Zlou2 = HyperLouvain(H,kmax,Ω)

mlou = modularity(H,Zlou,Ω)
mlou2 = modularity(H,Zlou2,Ω)

mlou ≈ mlou2


Louvain Iteration 1
Louvain Iteration 2
  1.780899 seconds (7.33 M allocations: 584.272 MiB, 12.07% gc time)

Took 7.700920104980469e-5 seconds to convert to different hypergraph formats
Louvain Iteration 1
Louvain Iteration 2
  1.093634 seconds (5.54 M allocations: 481.650 MiB, 18.19% gc time)


true

In [35]:
# Also compare modularity scores against the ground truth community,
# and all singleton clusters
# Strange that trivially putting all nodes in one cluster does better than the ground truth...
# Am I missing something?
Zsing = collect(1:n)
msing = modularity(H,Zsing,Ω)
mtrue = modularity(H,Z,Ω)


println("$msing\n $mtrue\n $mlou")

-1586.730331297104655748610860309215364605250897511723451316356658935546875
 -1741.071008800072559990926727492045744671855800334014929831027984619140625
 -1586.629202430686990335236580154480712234743577937479130923748016357421875
